In [11]:
import os
import ast
import pickle
import time
from tqdm import tqdm
import json
from glob import glob
from collections import Counter, defaultdict
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np

from load_data import load_user_time_read, load_raw
from tfidf import TFIDFGenerator, get_df
from config import Config
from utils import iterate_data_files, squeeze, save_as_pickle
from preprocessing import PostIdEncoder
from utils import squeeze

In [2]:
dev = [open('../preprocessed/train', 'r').readlines()]

In [3]:
start = time.time()
user_time_read = load_user_time_read(root_dir='../preprocessed/user_time_read.json')
encoder = PostIdEncoder(root_dir='../encodings')
tfidf = TFIDFGenerator('../tfidf')
print(f'Loading time: {time.time() - start}')

Loading time: 18.295762062072754


In [4]:
train_start = '2018100100'
train_end = '2019022200'
dev_start = '2019022200'
dev_end = '2019030100'

# set user list
dev_user_list = []
for daily in dev:
    users = [u.split()[0] for u in daily]
    dev_user_list.extend(users)

In [25]:
def filter_read_by_time(history, start, end):
    history_filtered = list(filter(lambda x: (int(start) <= int(x[0].split('_')[0])) and (int(end) >= int(x[0].split('_')[-1])), history))
    return history_filtered

start = Config.train_start
end = Config.train_end
n_splits = 20000
batch_size = len(dev_user_list) // n_splits

for i in range(n_splits):
    print(f'Getting User Preferences of Batch #{i}...')
    if i == n_splits-1:
        dev_user_batch = dev_user_list[i*batch_size:]
    else:
        dev_user_batch = dev_user_list[i*batch_size:(i+1)*batch_size]

    posts = pd.DataFrame()
    user_preferences = np.zeros((7000, 1))

    for user_id in tqdm(dev_user_batch, desc=f'Extracting user prefereces based on {start}-{end}'):
        history = filter_read_by_time(user_time_read[user_id], start, end)
        history = squeeze(list(map(lambda x: x[-1], history)))
        
        user_tfidf = tfidf.generate(encoder.transform(history), drop_id=False)
        preference = user_tfidf.drop('post_meta_id', axis=1).values.sum(axis=0)

        posts = pd.concat([posts, user_tfidf], axis=0, ignore_index=True)
        user_preferences =np.hstack([user_preferences,preference.reshape(7000, 1)])

    print('Postprocessing...')
    user_preferences = user_preferences[:, 1:]
    posts = posts.drop_duplicates(ignore_index=True)
    post_meta_id = posts['post_meta_id'].tolist()
    posts = posts.drop('post_meta_id', axis=1)
    idf = np.log(tfidf.DF.values.squeeze()) - np.log((posts != 0).sum().values + 1e-4)
    break

Extracting user prefereces based on 2018100100-2019022200:  21%|██▏       | 3/14 [00:00<00:00, 26.26it/s]Getting User Preferences of Batch #0...
Extracting user prefereces based on 2018100100-2019022200: 100%|██████████| 14/14 [00:02<00:00,  6.86it/s]
Postprocessing...


In [6]:
def filter_read_by_time(history, start, end):
    history_filtered = list(filter(lambda x: (int(start) <= int(x[0].split('_')[0])) and (int(end) >= int(x[0].split('_')[-1])), history))
    return history_filtered

def calculate_export_user_preferences(start: str=Config.train_start, end: str=Config.train_end, n_splits: int=50, save_path: str='./offline_tasks/user_preferences'):
    print('Loading tools...', end='\t')
    dev_raw = [open('./preprocessed/train', 'r').readlines()]
    dev_user_list = []
    for daily in dev_raw:
        users = [u.split()[0] for u in daily]
        dev_user_list.extend(users)

    user_time_read = load_user_time_read(root_dir='./preprocessed/user_time_read.json')
    encoder = PostIdEncoder(root_dir='./encodings')
    tfidf = TFIDFGenerator('./tfidf')
    print('loaded!')

    batch_size = len(dev_user_list) // n_splits

    for i in range(n_splits):
        print(f'Getting User Preferences of Batch #{i}...')
        if i == n_splits-1:
            dev_user_batch = dev_user_list[i*batch_size:]
        else:
            dev_user_batch = dev_user_list[i*batch_size:(i+1)*batch_size]

        posts = pd.DataFrame()
        user_preferences = np.zeros((7000, 1))

        for user_id in tqdm(dev_user_batch, desc=f'Extracting user prefereces based on {start}-{end}'):
            history = filter_read_by_time(user_time_read[user_id], start, end)
            preference = np.zeros((1, 7000))
            for h in history:
                partial_tfidf = tfidf.generate(encoder.transform(h[-1]), drop_id=False)
                preference += partial_tfidf.drop('post_meta_id', axis=1).values.sum(axis=0)
                posts = pd.concat([posts, partial_tfidf], axis=0, ignore_index=True)
            user_preferences = np.hstack([user_preferences, preference.reshape(7000, 1)])

        print('Postprocessing...')
        user_preferences = user_preferences[:, 1:]
        posts = posts.drop_duplicates(ignore_index=True)
        post_meta_id = posts['post_meta_id'].tolist()
        posts = posts.drop('post_meta_id', axis=1)
        idf = np.log(tfidf.DF.values.squeeze()) - np.log((posts != 0).sum().values + 1e-4)

        print('Saving...')
        batch_name = f'({start}-{end})batch{i+1:0>2d}'
        os.mkdir(os.path.join(save_path, batch_name))
        save_npz(os.path.join(save_path, batch_name, f'posts{i+1:0>2d}.npz'), csr_matrix(posts.values))
        save_as_pickle(post_meta_id, os.path.join(save_path, batch_name, f'post_meta_id{i:0>2d}.pkl'))
        np.save(os.path.join(save_path, batch_name, f'idf{i+1:0>2d}.npy'), idf)
        np.save(os.path.join(save_path, batch_name, f'user_preferences{i+1:0>2d}.npy'), user_preferences)

Extracting user prefereces based on 2018100100-2019022200: 100%|██████████| 100/100 [46:26<00:00, 27.87s/it]
Postprocessing...


In [ ]:
os.

In [7]:
recommend_output = np.matmul((posts * idf).values, user_preferences)
enc = PostIdEncoder(root_dir='../encodings')

In [10]:
np.save('test_recommend.npy', recommend_output)

In [8]:
recommend_output_user = pd.DataFrame(dict(value=recommend_output[:, 1], id=post_meta_id)).sort_values(by='value', ascending=False)

In [52]:
recommend_output_user['id'] = recommend_output_user['id'].apply(lambda x: enc.inverse_transform(x))

In [53]:
recommend_output_user.head(100)

,value,id
39,322457.075150,@englishspeaking_66
56,313143.167282,@linecard_45
60,313143.167282,@linecard_43
57,313143.167282,@linecard_42
98,312260.888493,@linecard_77
...,...,...
103,188480.532344,@hygo92_85
82,184787.498120,@muncoach_25
206,184734.588007,@thepiano_114
131,184527.611553,@thepiano_106
